# *CoastSat*: Multi-beach download

## Initial settings

Refer to the **Installation** section of the README for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
import pytz
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects


## Retrieval of the images from GEE

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

To create you ROI polygon you can either:
- enter the coordinates manually as in the example below (making sure the first and last coordinate are the same);
- go to [geojson.io](https://geojson.io), draw a polygon for your area of interest and export it as a GeoJSON and then read it with the function `SDS_tools.polygon_from_geojson()`.
- go to Google MyMaps, draw a polygon and export it as kml and then read it with the function `SDS_tools.polygon_from_kml()`.

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided in Tier 1 and Tier 2, only Tier 1 images can be used for time-series analysis.

For Landsat, users can also choose between Collection 1 and Collection 2 with the `collection` variable. Note that from 1st Jan 2022 newly acquired Landsat images are only available in Collection 2, with Landsat 9 only available in Collection 2, so it's preferred that you use Collection 2.

There is also the option to specify which Sentinel-2 or Landsat tile to download the images from. This can be specified by using `inputs['S2tile'] = '56HLH'` (Sentinel-2 MGRS grid) or `inputs['LandsatWRS'] = '089083'` (Landsat WRS-2 grid). These values correspond to the tiles covering Sydney. You can view Sentinel-2 tiles [here](https://eatlas.org.au/data/uuid/f7468d15-12be-4e3f-a246-b2882a324f59) and Landsat tiles [here](https://www.usgs.gov/media/files/landsat-wrs-2-scene-boundaries-kml-file). Specifying the tile helps to remove duplicates from overlapping tiles, especially in Sentinel-2 where the overlap is significant.

In [2]:
# region of interest (longitude, latitude)
geojson_polygon = os.path.join(os.getcwd(), 'GIS', 'playa_punta_de_lobos.geojson')
polygon = SDS_tools.polygon_from_geojson(geojson_polygon)

# or load from .kml
# kml_polygon = os.path.join(os.getcwd(), 'GIS', 'playa_punta_de_lobos.geojson')
# polygon = SDS_tools.polygon_from_kml(kml_polygon)
# date range
dates = ['1985-01-01', '2024-12-31']
# satellite missions ['L5','L7','L8','L9','S2']
sat_list = ['L5', 'L7', 'L8','L9','S2']
# name of the site
sitename = 'PLAYA_PUNTA_DE_LOBOS'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list,
          'sitename': sitename, 'filepath':filepath}
# optionally, you can select your Sentinel-2 or Landsat tile to avoid duplicates from multiple tiles
# inputs['S2tile'] = '56HLH'
# inputs['LandsatWRS'] = '089083'
# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Number of images available between 1985-01-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 410 images
     L7: 863 images
     L8: 463 images
     L9: 121 images
     S2: 1950 images
  Total to download: 3807 images
L5: 410/410 images already exist, 0 to be downloaded
L7: 816/863 images already exist, 47 to be downloaded
L8: 463/463 images already exist, 0 to be downloaded
L9: 121/121 images already exist, 0 to be downloaded
S2: 1613/1950 images already exist, 334 to be downloaded
- In Landsat Tier 2 (not suitable for time-series analysis):
     L5: 112 images
     L7: 72 images
     L8: 78 images
  Total Tier 2: 262 images


The function `SDS_download.retrieve_images(inputs)` retrives the satellite images from Google Earth Engine.

By default, only Landsat Tier 1 Top-of-Atmosphere and Sentinel-2 Level-1C products are downloaded. 

You can decide to skip the Landsat 7 images affected by the May 2003 Scan-Line-Corrector issue by settings `inputs['skip_L7_SLC'] = True`, although this is not recommended as L7 images can still be processed with the gaps.

In case you need to access Tier 2 images for qualitative analysis, you need to set `inputs['include_T2'] = True` before calling `retrieve_images()`.

In [3]:
# option to skip L7 images affected by the Scan-Line-Correction error after 31st May 2003
# inputs['skip_L7_SLC'] = True
# option to include Tier 2 images (ONLY IF NO OTHER IMAGES ARE AVAILABLE)
# inputs['include_T2'] = True

In [4]:
metadata = SDS_download.retrieve_images(inputs)

GEE initialized (existing token).
Number of images available between 1985-01-01 and 2024-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 410 images
     L7: 863 images
     L8: 463 images
     L9: 121 images
     S2: 1950 images
  Total to download: 3807 images
L5: 410/410 images already exist, 0 to be downloaded
L7: 816/863 images already exist, 47 to be downloaded
L8: 463/463 images already exist, 0 to be downloaded
L9: 121/121 images already exist, 0 to be downloaded
S2: 1613/1950 images already exist, 334 to be downloaded
- In Landsat Tier 2 (not suitable for time-series analysis):
     L5: 112 images
     L7: 72 images
     L8: 78 images
  Total Tier 2: 262 images

L5: 0 images

L7: 47 images

L8: 0 images

L9: 0 images

S2: 334 images
100%
Satellite images downloaded from GEE and save in c:\Users\pedro\Documents\RAFA\CoastSat-RRG\data\PLAYA_PUNTA_DE_LOBOS


**If you have already retrieved the images**, just load the metadata file by only running the section below

In [38]:
metadata = SDS_download.get_metadata(inputs)